In [3]:
import pandas as pd 
%pip install openpyxl


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
xls= pd.ExcelFile("global_superstore_2016.xlsx")
xls.sheet_names

['Orders', 'Returns', 'People']

In [9]:
orders = pd.read_excel(xls, sheet_name="Orders", engine='openpyxl')
people = pd.read_excel(xls, sheet_name="People", engine='openpyxl')
returns = pd.read_excel(xls, sheet_name="Returns", engine='openpyxl')

In [11]:
print(orders.head())

   Row ID                  Order ID Order Date  Ship Date     Ship Mode  \
0   40098  CA-2014-AB10015140-41954 2014-11-11 2014-11-13   First Class   
1   26341    IN-2014-JR162107-41675 2014-02-05 2014-02-07  Second Class   
2   25330    IN-2014-CR127307-41929 2014-10-17 2014-10-18   First Class   
3   13524   ES-2014-KM1637548-41667 2014-01-28 2014-01-30   First Class   
4   47221   SG-2014-RH9495111-41948 2014-11-05 2014-11-06      Same Day   

    Customer ID     Customer Name      Segment  Postal Code           City  \
0  AB-100151402     Aaron Bergman     Consumer      73120.0  Oklahoma City   
1     JR-162107     Justin Ritter    Corporate          NaN     Wollongong   
2     CR-127307      Craig Reiter     Consumer          NaN       Brisbane   
3    KM-1637548  Katherine Murray  Home Office          NaN         Berlin   
4    RH-9495111       Rick Hansen     Consumer          NaN          Dakar   

   ...   Product ID    Category Sub-Category  \
0  ...  TEC-PH-5816  Technology 

In [12]:
print(people.head())

              Person           Region
0  Marilène Rousseau        Caribbean
1   Andile Ihejirika   Central Africa
2  Nicodemo Bautista  Central America
3     Cansu Peynirci     Central Asia
4         Lon Bonher       Central US


In [15]:
orders['Order Date']= pd.to_datetime(orders['Order Date'],errors='coerce')
orders['Ship Date'] = pd.to_datetime(orders['Ship Date'], errors='coerce')

# Convert Sales and Profit to numeric (in case they're stored as text)
orders['Sales'] = pd.to_numeric(orders['Sales'], errors='coerce')
orders['Profit'] = pd.to_numeric(orders['Profit'], errors='coerce')


In [16]:
len(orders)

51290

In [17]:
orders.dropna(subset=['Order ID','Order Date','Sales','Profit'],inplace=True)

In [18]:
len(orders)

51290

In [24]:
order_people = orders.merge(people,how='right',on='Region')
print(order_people.head())
print(len(order_people))

   Row ID                 Order ID Order Date  Ship Date       Ship Mode  \
0   220.0  US-2012-RR1952536-41270 2012-12-27 2012-12-29    Second Class   
1  2641.0  US-2012-LH1675036-41129 2012-08-08 2012-08-08        Same Day   
2   879.0  MX-2015-TM2149031-42329 2015-11-21 2015-11-21        Same Day   
3  5306.0  MX-2015-TB2125031-42289 2015-10-12 2015-10-15     First Class   
4  6019.0  MX-2013-JF1529531-41564 2013-10-17 2013-10-21  Standard Class   

  Customer ID   Customer Name    Segment  Postal Code              City  ...  \
0  RR-1952536       Rick Reed  Corporate          NaN     Santo Domingo  ...   
1  LH-1675036    Larry Hughes   Consumer          NaN     Santo Domingo  ...   
2  TM-2149031   Tony Molinari   Consumer          NaN          Camagüey  ...   
3  TB-2125031    Tim Brockman   Consumer          NaN  Santiago de Cuba  ...   
4  JF-1529531  Jason Fortune-   Consumer          NaN        Guantánamo  ...   

     Category Sub-Category                                 Pro

In [29]:
order_people.to_csv("cleaned_orders.csv",index=False)

In [30]:
import sqlite3

In [31]:
conn = sqlite3.connect("retail_project.db")

In [32]:
orders.to_sql("Orders",conn)

ValueError: Table 'Orders' already exists.

In [34]:
orders.to_sql("Orders", conn, if_exists="replace", index=False)
people.to_sql("People", conn, if_exists="replace", index=False)
returns.to_sql("Returns", conn, if_exists="replace", index=False)
order_people.to_sql("OrdersPeople", conn, if_exists="replace", index=False)


50908